In [5]:
import pandas as pd
import json
import os

In [6]:
def load_game(d):
    with open(os.path.join(d, "game.json")) as f:
        game = json.load(f)
    with open(os.path.join(d, "settings.json")) as f:
        settings = json.load(f)
    return game, settings

In [11]:
settings

{'slpVersion': '2.0.1',
 'isTeams': False,
 'isPAL': False,
 'stageId': 2,
 'players': [{'playerIndex': 0,
   'port': 1,
   'characterId': 2,
   'characterColor': 3,
   'startStocks': 4,
   'type': 0,
   'teamId': 0,
   'controllerFix': 'UCF',
   'nametag': ''},
  {'playerIndex': 3,
   'port': 4,
   'characterId': 15,
   'characterColor': 3,
   'startStocks': 4,
   'type': 0,
   'teamId': 0,
   'controllerFix': 'UCF',
   'nametag': ''}],
 'winner': False,
 'jig_id': 3,
 'other_id': 0}

In [14]:
setting_features = ['stageId', 'winner']
game_features = {
    'pre': [], 
    'post': ['positionX', 'positionY', 'facingDirection', 
             'percent', 'shieldSize', 'stocksRemaining', 'lCancelStatus']
}
in_dir = "./out"
out_dir = "./data"
os.makedirs(os.path.join(out_dir), exist_ok=True)
for d in os.listdir(in_dir):
    
    game, settings = load_game(os.path.join(in_dir, d))
    jig_id = settings['jig_id']
    other_id = settings['other_id']
    setting_data = [settings[feat] for feat in setting_features]
    data = []
    for frame in game:
        frame_data = []
        frame_data += [int(frame)]
        for k in game_features:
            frame_data += [game[frame]['players'][jig_id][k][feat] for feat in game_features[k]]
        for k in game_features:
            frame_data += [game[frame]['players'][other_id][k][feat] for feat in game_features[k]]
        frame_data += setting_data

        data.append(frame_data)

    columns = ['frame']
    for k in game_features:
        columns += ["p1_{}_{}".format(k, feat) for feat in game_features[k]]
    for k in game_features:
        columns += ["p2_{}_{}".format(k, feat) for feat in game_features[k]]
    columns += setting_features
    
    
    pd.DataFrame(data, columns=columns).to_csv(os.path.join(out_dir, d + ".csv"), index=False)